# Creation  df  au cas ou 

In [ ]:
df = pd.read_pickle('df')
df.shape
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df = df[:1017100]
df.shape

In [ ]:
f = int(len(df)/10)
f
for x in range(10):    
    dfx = df[x*f : (x+1)*f]
    x*f , (x+1)*f , dfx.shape
    dfx.to_csv('valeursfoncieres-2019_{}.csv'.format(x))

In [ ]:
df.shape

# New

Sur le site de data gouv : https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres-geolocalisees/
On récupère plusieurs fichiers correspondants à différentes. Dans cette section on présente une routine pour stocker les chemins d’acces de ces fichiers et les stocker dans une liste grâce à l’utils glob. Cette démarche est similaire une request par exemple.

1. Glob permet de créer une liste python des fichiers .txt stockés dans le dossier data.
1. On charge le premier élément de la liste dans un dataframe  grâce à l’option read_csv. 
1. Il est important après avoir chargé des données de vérifier son shape (nombre de ligne / colonne) avec la commande du même nom mais également de print un échantillon, ici les 10 premières lignes avec la commande head équivalent a .loc[ :10]. 
1. On note la présence de valeur null, donc non renseigné mais également de nombreuses colonnes dont seulement une partie seront utile par la suite.
1. Pour chaque section on sauvegarde le dataframe dans le format le plus efficace pour des df pandas : pickle.


## import   

In [ ]:
import json
import time
import glob, os
import random

import pandas as pd
import numpy as np

from IPython.display import display, HTML
import matplotlib.pyplot as plt

#permet d'afficher des graph dans le notebook
%matplotlib inline 

#permet d'afficher plusieurs sortis par cellule sans utiliser print()
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#permet d'augemnter la largeur des cellules
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70%! important; }</style>"))

In [ ]:
#optionnel si difficulté pour autocompletion 
%config Completer.use_jedi = False

## Load [exo]
    charger un dataframe a partir de plusieurs csv

In [ ]:
Filelist = ['Data2\\valeursfoncieres-2019_{}.csv'.format(x) for x in range(5)]

In [ ]:
path = 'Data2\*.csv'
Filelist = glob.glob(path)
Filelist

In [ ]:
listdf = [pd.read_csv(f) for f in Filelist]

In [ ]:
df  = pd.concat(listdf)
df.shape

In [ ]:
df.shape
df.head()

In [ ]:
df = df.drop(columns='Unnamed: 0')

# nettoyage data

Il est fréquent de vérifier le format des données en particulier si une colonne contient différents types (ex int , str ….), il convient de systématiquement corriger le format des données pour pouvoir les analyser avec pandas.
1. On présente ici une méthode rapide à mettre en œuvre pour établir un état des lieux de notre dataframe. La fonction apply de pandas permet d’appliquer une fonction à toutes les valeurs d’une colonne dans ce cas  la fonction python type. On note que 2 colonnes sont concernées par des types différents Valeur_fonciere & Code_departement (str & int).  
1. Commençons par Valeur_fonciere, on réalise un masque des données de type str. Ces valeurs procèdent en plus une virgule, il faut le corriger. Nous appliquons un split sur ces valeurs en sélectionnant la partie à gauche de la virgule et assignons ces valeurs dans le dataframe, désormais nous pouvons sans risque utiliser la fonction pandas to_numeric pour passer du type str a float. 
1. Pour le Code_departement, un problème diffèrent se pose, comme on peut le voir en réalisant un print des valeurs uniques. Les codes département de la corse sont 2A et 2B, il faut donc conservé les valeurs sous un format str, de plus sur les valeurs inferieurs a 10 il faut rajouter le 0 ex 1 = 01.
1. La première étape est de passer toutes les valeurs en str avec la fonction astype (str) 
1. Ensuite réaliser un masque sur les valeurs de code inferieur à 10, donc avec seulement un chiffre, ici la fonction apply len renvoi le nombre de caractère pour chaque valeur, il suffit ensuite de faire un slice des données avec un mask len = 1  et d’ajouter le 0.
1. Dans le cas de la colonne de date pandas réalise cette opération avec la fonction to_datetime (attention toute fois au format des données d’entrée) 

## astuce print

In [ ]:
df.Valeur_fonciere.apply(type).value_counts()

In [ ]:
dftype = pd.DataFrame([df[c].apply(type).value_counts() for c in df.columns])
dftype

In [ ]:
dftype[dftype.isnull().sum(1) !=2]
dftype[dftype.isnull().sum(1) !=2].index

## Valeur_fonciere [exo]

In [ ]:
df.Valeur_fonciere.apply(type).value_counts()

In [ ]:
s = pd.Series(list(range(7)), index= list('ABCDEFG'))
mask = s>3
s[mask]

In [ ]:
s = pd.Series(list('0123456'), index= list('ABCDEFG'))
s.values
pd.to_numeric(s)

In [ ]:
mask = df.Valeur_fonciere.apply(type) == str
df.Valeur_fonciere[mask]

In [ ]:
new_val = df.Valeur_fonciere[mask].str.split(',').str[0]
new_val

In [ ]:
df.loc[mask, 'Valeur_fonciere'] = new_val
df.Valeur_fonciere = pd.to_numeric(df.Valeur_fonciere)

In [ ]:
df.Valeur_fonciere.apply(type).value_counts()

## Code_departement [exo]

In [ ]:
df.Code_departement.apply(type).value_counts()
df.Code_departement.unique()

In [ ]:
df.Code_departement = df.Code_departement.astype(str) 
df.Code_departement.unique()

In [ ]:
df.Code_departement = df.Code_departement.astype(str) 

In [ ]:
Len = df.Code_departement.apply(len)
Len.value_counts()

In [ ]:
mask = Len == 1
df.loc[mask, 'Code_departement']= '0' + df.loc[mask, 'Code_departement']
df.Code_departement.unique()

In [ ]:
df.Code_departement.nunique()

### Date_mutation

In [ ]:
df.Date_mutation = pd.to_datetime(df.Date_mutation)

In [ ]:
df.isnull().sum(1)

In [ ]:
df = df.fillna(method = 'ffill')

### Save

In [ ]:
df.to_pickle('df')

# Enrichissement : region  [exo]

L’enrichissement permet d’ajouter au dafaframe des colonnes de données utiles mais non présent dans le jeu de base. Ici nous allons rajouter les régions
1. la csv des donnés est dans le lien suivant : https://www.data.gouv.fr/fr/datasets/regions-departements-villes-et-villages-de-france-et-doutre-mer/
1. On charge ces données dans un nouveau dataframe dfgeo et ainsi refaire les étapes de nettoyage des données avant de le combiner avec df.
1. Apres comptage il n’y a que 100 dep dans ce fichier et en comparant les code_departement dans les deux fichiers il manque le dep 75, on peut le rajouter avec un dict en entrée.
1. Enfin on ajoute les colonnes région et code_region dans df avant de sauvegarder.

In [ ]:
df = pd.read_pickle('df')
df.shape

In [ ]:
df[df.Code_departement.isnull()]

In [ ]:
df.head()

In [ ]:
dfgeo = pd.read_csv('data/departements-france.csv')
dfgeo
dfgeo.shape

In [ ]:
#Verification nombre de departement
dfgeo.code_departement.nunique()

In [ ]:
#ex map
s = pd.Series(['a','b','c'])
d ={'a' : 0,
    'b' : 1,
    'c' : 2}
s.map(d)
s.map(pd.Series(d))

In [ ]:
df.Code_departement.unique()

In [ ]:
df[~df.Code_departement.isin(dfgeo.code_departement)].Code_departement.unique()

In [ ]:
dfgeo.loc[dfgeo.shape[0]] = ['75', 'Paris', '11', 'Île-de-France']

In [ ]:
dfgeo[dfgeo.nom_region == 'Île-de-France']

In [ ]:
dfgeo.code_departement.nunique()

In [ ]:
df['Region'] = df.Code_departement.map(dfgeo.set_index('code_departement')['nom_region'])
df['code_region'] = df.Code_departement.map(dfgeo.set_index('code_departement')['code_region'])

In [ ]:
df.to_pickle('df')

# json export


In [ ]:
#PL un des points importants: la création/génération (et également l'utilisation de fait) de format .json.

params = ['split', 'records', 'index', 'columns', 'values', 'table']

for param in params:
    print (param)
    print (df.iloc[:5,:5].to_json(orient=param))
    print ()

## Test export json 

https://app.rawgraphs.io/

In [ ]:
###rawgraphs (~Maxx) Optionnel

data.index.name = 'index'

data[:1000].fillna(0).astype(int).to_clipboard(sep=',')
